In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

quotes_url = 'https://quotes.toscrape.com/page/'

quotes = []
authors = []
tags_list = []
born_info = []
descriptions = []

page_num = 1

while True:
    response = requests.get(quotes_url + str(page_num))
    soup = BeautifulSoup(response.text, 'html.parser')
    
    quote_divs = soup.find_all('div', class_='quote')
    
    if not quote_divs:
        break

    for quote in quote_divs:
        text = quote.find('span', class_='text').text
        author = quote.find('small', class_='author').text
        tags = [tag.text for tag in quote.find_all('a', class_='tag')]
        
        quotes.append(text)
        authors.append(author)
        tags_list.append(tags)

        author_url = quote.find('a')['href']
        author_page = requests.get('https://quotes.toscrape.com' + author_url)
        author_soup = BeautifulSoup(author_page.text, 'html.parser')
        born = author_soup.find('span', class_='author-born-date').text
        born += ', ' + author_soup.find('span', class_='author-born-location').text
        description = author_soup.find('div', class_='author-description').text.strip()
        
        born_info.append(born)
        descriptions.append(description)

    page_num += 1
    
df = pd.DataFrame({
    'Quote': quotes,
    'Author': authors,
    'Tags': tags_list,
    'Born': born_info,
    'Description': descriptions
})

df.to_csv('quotes.csv', index=False)
print("Data saved to 'quotes.csv'")